In [1]:
#Module imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
import torch.nn.functional as F

/data/rsg/chemistry/johnyang/miniconda3/envs/s2s/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# embeddings = torch.load('../data/esmvalset2.pt')

In [3]:
dataset_folder = "../google_prot_fns/"
filters = 64
epochs = 60
batch_size = 128
protein_len = 200

### As a toy example we only select the largest 2 protein families

In [4]:
protein_family_number = 2

In [5]:
def read_data(name_sub_folder):
  full_data = []
  for f in os.listdir(os.path.join(dataset_folder, name_sub_folder)):
    data = pd.read_csv(os.path.join(dataset_folder, name_sub_folder, f))
    full_data.append(data)
  return pd.concat(full_data)  

In [6]:
dataset_train = read_data("train")
dataset_train.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,HTH_12,D7DT16_METV3/7-72,PF08461.10,IAILNLLSKYDKPIGAKIIAEDLKLR.GYDIGERAVRYHLQSMDDD...,IAILNLLSKYDKPIGAKIIAEDLKLRGYDIGERAVRYHLQSMDDDE...
1,NFACT-R_1,D6GU87_PARA5/390-500,PF05670.13,KFRFFS...............TSLN......KLCIIGKDVNQ.NES...,KFRFFSTSLNKLCIIGKDVNQNESLIQKHAEKGDIVGHADVFGSPF...
2,Nop53,B2ASQ0_PODAN/21-405,PF07767.11,SRKGKKAWRKNVDVT...DITKGLDQLN.TQKIIGG.VIAEKDSAD...,SRKGKKAWRKNVDVTDITKGLDQLNTQKIIGGVIAEKDSADLFVLD...
3,DUF72,D3SM48_THEAH/20-240,PF01904.18,YPPRLRR.DE.WIKYYAKH..FVVLELNATF........YG.FPDR...,YPPRLRRDEWIKYYAKHFVVLELNATFYGFPDRGSIKSMLSRVKDL...
4,Glyco_hydro2_C5,U2B330_9CLOT/643-745,PF18565.1,LRLTTDKEQLSADGKSLAFLTITAVDREGRTVENAVNRILVKVEGA...,LRLTTDKEQLSADGKSLAFLTITAVDREGRTVENAVNRILVKVEGA...


In [7]:
dataset_test = read_data("test")
dataset_test.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,HD,Q9X1N3_THEMA/52-163,PF01966.22,RFAHSLGTMHVAGLYARNLFK......................ESD...,RFAHSLGTMHVAGLYARNLFKESDRIRIVRLAALLHDVGHGPFSHQ...
1,ChW,K4QXH7_STRDJ/317-351,PF07538.11,HLSGSG........W.........Q.......K.....P..V.CDG...,HLSGSGWQKPVCDGTLAGTTGQEKPIKAINIAVTG
2,7TM_GPCR_Srd,A8WZJ1_CAEBR/10-305,PF10317.9,LSIFYPIFFVITLIAQLFLMFLIIKHSPKSIHMLRIILGLTCIFQI...,LSIFYPIFFVITLIAQLFLMFLIIKHSPKSIHMLRIILGLTCIFQI...
3,Integrin_beta,W5MQ40_LEPOC/123-370,PF00362.18,TEDYPVDVYYLMDLSASMSDDL.....EMIKDLGSTLSRE..MAKL...,TEDYPVDVYYLMDLSASMSDDLEMIKDLGSTLSREMAKLTSKFRLG...
4,Rimk_N,A0A0U2RI95_9ALTE/1-94,PF18030.1,MRIAVLSRNANL..YSTRRLKEEGEAHGHEMHIIDPLKCYMNINMQ...,MRIAVLSRNANLYSTRRLKEEGEAHGHEMHIIDPLKCYMNINMQSS...


In [8]:
dataset_dev = read_data("dev")
dataset_dev.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,ASL_C,K4A843_SETIT/406-520,PF08328.11,SRMQRDLTDSTVLRNLGVGLGHSLLAYKATMRGISKVQVNESRLAE...,SRMQRDLTDSTVLRNLGVGLGHSLLAYKATMRGISKVQVNESRLAE...
1,DUF1735,E7RLQ4_9BACT/29-153,PF08522.10,YKH...V............FSFI.............CN.TD........,YKHVFSFICNTDHVSQKMFNLSDTLRTGYISLSMGGSNPTDEDVTV...
2,DUF4846,R5X9T4_9CLOT/55-294,PF16138.5,IKERYDVPEGYERVSTEENSFGEFLRNSKLEKYGEKVKYYDGRT.....,IKERYDVPEGYERVSTEENSFGEFLRNSKLEKYGEKVKYYDGRTKD...
3,Candida_ALS,A5DJ08_PICGU/1856-1887,PF05792.13,TTLT.STWN.....GT...V..TTT....TT.FS...ASV.........,TTLTSTWNGTVTTTTTFSASVGGTETVIVEIP
4,Seipin,B6H3Z9_PENRW/52-254,PF06775.14,TFLIGISAIAYGVF.YFNFIP.TVG.LEREVHLQF..G...........,TFLIGISAIAYGVFYFNFIPTVGLEREVHLQFGDGNPWGTAHFDSE...


In [9]:
dataset_train['family_accession'].value_counts()

PF13649.6     3637
PF00560.33    1927
PF13508.7     1761
PF06580.13    1537
PF02397.16    1528
              ... 
PF17055.5        1
PF09096.10       1
PF10894.8        1
PF03507.13       1
PF17606.2        1
Name: family_accession, Length: 17929, dtype: int64

In [10]:
all_datasets = pd.concat([dataset_train, dataset_test, dataset_dev])

In [11]:
classes = all_datasets['family_accession'].value_counts()[:protein_family_number].index

In [12]:
def return_from_dataset(dataset, classes):
  return [dataset.loc[dataset['family_accession'].isin(classes)].reset_index(), classes]

In [13]:
[toy_dataset, classes] = return_from_dataset(all_datasets, classes)

In [14]:
# toy_dataset.to_csv("../data/2_class.csv")

We reset to save memory. Above is an example for loading the data.

In [15]:
%reset

Nothing done.


In [16]:
#Module imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
import torch.nn.functional as F

In [17]:
dataset_folder = "../google_prot_fns/"
filters = 64
epochs = 60
batch_size = 128
protein_len = 200

In [18]:
toy_dataset = pd.read_csv('../data/2_class.csv')
classes = pd.unique(toy_dataset['family_accession'])

In [19]:
len(classes) #Should be 2

2

In [20]:
# Irrelevant. Example. We will use ESMFnDataset defined below.
class SeqFnDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, classes):
        self.dataset = dataset
        self.classes = classes
        self.class_to_idx = {classes[i]: i for i in range(len(classes))}
        self.idx_to_class = {i: classes[i] for i in range(len(classes))}
        self.letter_to_num = {'C': 4, 'D': 3, 'S': 15, 'Q': 5, 'K': 11, 'I': 9,
                'P': 14, 'T': 16, 'F': 13, 'A': 0, 'G': 7, 'H': 8,
                'E': 6, 'L': 10, 'R': 1, 'W': 17, 'V': 19, 
                'N': 2, 'Y': 18, 'M': 12}
        self.num_to_letter = {v:k for k, v in self.letter_to_num.items()}
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        row = self.dataset.iloc[idx]
        seq = torch.as_tensor([self.letter_to_num[a] for a in row['sequence']],
                    device='cpu', dtype=torch.long)
        class_idx = torch.tensor(self.class_to_idx[row['family_accession']])
        label = F.one_hot(class_idx, num_classes=len(self.classes))
        return seq, label


In [21]:
dl = torch.utils.data.DataLoader(SeqFnDataset(toy_dataset, classes))

In [22]:
for batch in dl:
    print(batch)
    break

[tensor([[19, 10,  3,  9,  0,  7,  7, 16,  7,  3, 19, 16, 13, 16, 19,  0,  1, 16,
         11,  8, 14,  1,  8,  9,  5,  4, 16,  3, 10, 19, 14,  6, 12, 10,  3, 19,
          0,  1, 12,  8,  9,  3,  6,  7,  3,  7,  0,  7, 19, 14,  9,  3, 13,  6,
         19, 19,  3,  0,  5,  2,  9, 14, 18,  6,  3,  0, 15, 18,  3, 19,  9, 16,
         12,  0, 18,  7,  9,  1,  2, 12, 14,  3,  1, 14,  1,  0, 10, 15,  6, 12,
         13,  1, 19, 10, 11, 14,  7,  7]]), tensor([[1, 0]])]


In [23]:
import esm
from esm.model.esm2 import ESM2

In [24]:
model_data = torch.load('/data/rsg/chemistry/johnyang/home/pt_models/esm2_t33_650M_UR50D.pt')
regression_data = torch.load('/data/rsg/chemistry/johnyang/home/pt_models/esm2_t33_650M_UR50D-contact-regression.pt')

In [25]:
from utils.ls_utils import *

In [26]:
from utils.gpu_utils import *

In [27]:
chosen_gpu = get_free_gpu()
device = 'cuda'

cuda
Using GPUs: 0


In [28]:
esm_model, alphabet = load_model_and_alphabet_core("esm2_t33_650M_UR50D", model_data, regression_data)
batch_converter = alphabet.get_batch_converter()
esm_model.to(device)
esm_model.eval()
print('done')

done


In [29]:
for row in toy_dataset.iterrows():
    seq = row[1]['sequence']
    print(row)
    print(seq)
    break

(0, Unnamed: 0                                                          0
index                                                             910
family_id                                             Methyltransf_25
sequence_name                                     S6CDL6_9ACTN/62-159
family_accession                                            PF13649.6
aligned_sequence    VLDIAGGT.G...D..VT...FT..VA.RT...........KH......
sequence            VLDIAGGTGDVTFTVARTKHPRHIQCTDLVPEMLDVARMHIDEGDG...
Name: 0, dtype: object)
VLDIAGGTGDVTFTVARTKHPRHIQCTDLVPEMLDVARMHIDEGDGAGVPIDFEVVDAQNIPYEDASYDVITMAYGIRNMPDRPRALSEMFRVLKPGG


In [30]:
def pad_seq_to_max_len(seq, max_len):
    padding_token = '<pad>'
    seq = seq + padding_token * (max_len - len(seq))
    return seq

We want to pad sequences to all be <= a defined length.

In [31]:
data = [(row[1]['sequence_name'], pad_seq_to_max_len(row[1].sequence, protein_len)) for row in toy_dataset.iterrows()]

In [32]:
len(data)

6952

In [33]:
import tqdm

KEY AREA TO IMPROVE:

Here we save the embeddings in a directory ran one-by-one... This is extremely inefficient given we have 1 million sequences... \

The below script will extract embeddings given a Fasta file efficiently.

python esm/scripts/extract.py esm2_t33_650M_UR50D examples/data/some_proteins.fasta \
    examples/data/some_proteins_emb_esm2 --repr_layers 33 --include per_tok

You can write Fasta files from pandas dataframes easily. Here's an example: https://bootstrappers.umassmed.edu/guides/main/r_writeFasta.html

In [35]:
'''
RAN ALREADY FOR TOY EXAMPLE

for i, protein in tqdm.tqdm(enumerate(data)):
    batch_labels, batch_strs, batch_tokens = batch_converter([protein])
    batch_tokens = batch_tokens.to(device)
    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[33], return_contacts=False)
    embeddings = results['representations'][33].detach().cpu().numpy()
    sequence_name = protein[0]
    # Modify all slashes in sequence name to dashes or sees name as directory, perhaps this is a good thing...
    sequence_name = sequence_name.replace('/', '-')
    torch.save(embeddings, f'../data/toy_esm_embeddings/{sequence_name}.pt')
'''

"\nRAN ALREADY FOR TOY EXAMPLE\n\nfor i, protein in tqdm.tqdm(enumerate(data)):\n    batch_labels, batch_strs, batch_tokens = batch_converter([protein])\n    batch_tokens = batch_tokens.to(device)\n    with torch.no_grad():\n        results = esm_model(batch_tokens, repr_layers=[33], return_contacts=False)\n    embeddings = results['representations'][33].detach().cpu().numpy()\n    sequence_name = protein[0]\n    # Modify all slashes in sequence name to dashes or sees name as directory, perhaps this is a good thing...\n    sequence_name = sequence_name.replace('/', '-')\n    torch.save(embeddings, f'../data/toy_esm_embeddings/{sequence_name}.pt')\n"

In [38]:
sequence_name = row[1]['sequence_name']
sequence_name = sequence_name.replace('/', '-')

In [49]:
sequence_name

'S6CDL6_9ACTN-62-159'

In [39]:
ex = torch.load(f'../data/toy_esm_embeddings/{sequence_name}.pt')

In [40]:
ex.shape

(1, 202, 1280)

In [41]:
class ESMFnDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, classes, device='cuda', max_len=200):
        self.dataset = dataset
        self.classes = classes
        self.max_len = max_len
        self.device = device
        self.class_to_idx = {classes[i]: i for i in range(len(classes))}
        self.idx_to_class = {i: classes[i] for i in range(len(classes))}
        self.letter_to_num = {'C': 4, 'D': 3, 'S': 15, 'Q': 5, 'K': 11, 'I': 9,
                'P': 14, 'T': 16, 'F': 13, 'A': 0, 'G': 7, 'H': 8,
                'E': 6, 'L': 10, 'R': 1, 'W': 17, 'V': 19, 
                'N': 2, 'Y': 18, 'M': 12}
        self.num_to_letter = {v:k for k, v in self.letter_to_num.items()}
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        row = self.dataset.iloc[idx]
        sequence_name = row['sequence_name']
        sequence_name = sequence_name.replace('/', '-')
        embeddings = torch.load(f'../data/toy_esm_embeddings/{sequence_name}.pt')
        embeddings = torch.tensor(embeddings, device='cuda', dtype=torch.float32)

        '''Pad embeddings to max_len with zero vector'''
        if embeddings.size(1) < self.max_len:
            B, N, h = embeddings.size()
            pad = torch.zeros((B, self.max_len - embeddings.shape[1], h), device=self.device)
            embeddings = torch.cat((embeddings, pad), dim=1)

        class_idx = torch.tensor(self.class_to_idx[row['family_accession']])
        label = F.one_hot(class_idx, num_classes=len(self.classes))
        return embeddings, label

In [42]:
dataset = ESMFnDataset(toy_dataset, classes, device)

In [43]:
esmdl = torch.utils.data.DataLoader(dataset)

In [44]:
for batch in esmdl:
    print(batch[0][0])
    break

tensor([[[ 0.0243,  0.0271, -0.0837,  ..., -0.2799,  0.1995,  0.0577],
         [-0.0919, -0.0847, -0.0205,  ..., -0.0418,  0.1840, -0.1400],
         [ 0.1741,  0.1002,  0.0078,  ...,  0.0821,  0.0727, -0.0905],
         ...,
         [-0.0306, -0.0665,  0.0843,  ..., -0.0120, -0.0695, -0.1188],
         [-0.0623,  0.0503,  0.0476,  ...,  0.0348, -0.0654, -0.0851],
         [-0.0286,  0.0321,  0.0917,  ..., -0.0120, -0.0180, -0.0668]]],
       device='cuda:0')


In [45]:
class TransformerEncoder(torch.nn.Module):
    
    def __init__(self, input_size=1280, nheads=8, num_layers=6, device='cuda', classes=2, max_len=202, **kwargs):
        super(TransformerEncoder, self).__init__()
        self.input_size = input_size
        self.classes = classes
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=self.input_size, nhead=nheads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc1 = torch.nn.Linear(self.input_size, 256)
        self.fc2 = torch.nn.Linear(256, 32)
        self.fc3 = torch.nn.Linear(32 * max_len, self.classes)
        self.softmax = torch.nn.Softmax(dim=-1)
        self.device = device
        
    def forward(self, embedding):
        if len(embedding.size()) > 3 and embedding.size(0) == 1:
            embedding = embedding.squeeze(0)
            assert len(embedding.size()) == 3, 'Embedding has greater than 4 dimensions'
            
        B, N, h = embedding.shape
        hidden = self.transformer_encoder(embedding)
        hidden = self.fc1(hidden)
        hidden = self.fc2(hidden)
        '''Flatten hidden state'''
        hidden = hidden.view(B, -1)
        hidden = self.fc3(hidden)
        hidden = self.softmax(hidden)
        return hidden

In [46]:
model = TransformerEncoder(input_size=1280, nheads=8, num_layers=6, device=device, classes=len(classes)).to(device)

In [47]:
len(classes)

2

In [48]:
for batch in esmdl:
    embedding, label = batch
    pred = model(embedding)
    print(pred.shape) # shape is [batch_size, number of classes]
    break

torch.Size([1, 2])


# Learning to Split

Having trouble running Learning to Split right now because of environment issues.
Regardless you can focus on getting ESMFnDataset to work will all sequences.

Honestly, I don't know if it's possible to run both this Transformer model on 1 million sequences nor Learning to Split but it seems like if there's anyone who can figure it out, it's you.

In [ ]:
import ls

: 

In [ ]:
# python scripts/extract.py esm2_t33_650M_UR50D examples/data/some_proteins.fasta \
#   examples/data/some_proteins_emb_esm2 --repr_layers 33 --include per_tok

: 